In [1]:
# import modules
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv

In [13]:
# Try multiplying the process for each position group

url_dict = {
    "QB": "https://www.fantasypros.com/nfl/projections/qb.php?week=draft",
    "RB": "https://www.fantasypros.com/nfl/projections/rb.php?week=draft&scoring=PPR&week=draft",
    "WR": "https://www.fantasypros.com/nfl/projections/wr.php?week=draft&scoring=PPR&week=draft",
    "TE": "https://www.fantasypros.com/nfl/projections/te.php?week=draft&scoring=PPR&week=draft",
    "FLEX": "https://www.fantasypros.com/nfl/projections/flex.php?week=draft&scoring=PPR&week=draft",
    "K": "https://www.fantasypros.com/nfl/projections/k.php?week=draft",
    "DEF": "https://www.fantasypros.com/nfl/projections/dst.php?week=draft",
}

# Step 2: Loop through each key-value pair in the dictionary
for name, url in url_dict.items():
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Example: Locate and extract data from a table (customize this as needed)
    table = soup.find("table")

    if table:
        headers = [header.get_text().strip() for header in table.find_all("th")]
        rows = []
        for row in table.find_all("tr"):
            cells = [cell.get_text().strip() for cell in row.find_all("td")]
            if cells:  # Only append non-empty rows
                rows.append(cells)

        # Combine headers and rows into a DataFrame
        df = pd.DataFrame(rows, columns=headers)

        # Step 3: Save each DataFrame to a separate CSV file with the variable name
        filename = os.path.join("data/scraped", f"{name}_projections.csv")
        df.to_csv(filename, index=False)

        print(f"Data from {name} ({url}) saved to {filename}")

print("Scraping and data saving completed.")

Data from QB (https://www.fantasypros.com/nfl/projections/qb.php?week=draft) saved to data/scraped/QB_projections.csv
Data from RB (https://www.fantasypros.com/nfl/projections/rb.php?week=draft&scoring=PPR&week=draft) saved to data/scraped/RB_projections.csv
Data from WR (https://www.fantasypros.com/nfl/projections/wr.php?week=draft&scoring=PPR&week=draft) saved to data/scraped/WR_projections.csv
Data from TE (https://www.fantasypros.com/nfl/projections/te.php?week=draft&scoring=PPR&week=draft) saved to data/scraped/TE_projections.csv
Data from FLEX (https://www.fantasypros.com/nfl/projections/flex.php?week=draft&scoring=PPR&week=draft) saved to data/scraped/FLEX_projections.csv
Data from K (https://www.fantasypros.com/nfl/projections/k.php?week=draft) saved to data/scraped/K_projections.csv
Data from DEF (https://www.fantasypros.com/nfl/projections/dst.php?week=draft) saved to data/scraped/DEF_projections.csv
Scraping and data saving completed.


In [2]:
# URLs to scrape from

QB = "https://www.fantasypros.com/nfl/projections/qb.php?week=draft"
RB = "https://www.fantasypros.com/nfl/projections/rb.php?week=draft&scoring=PPR&week=draft"
WR = "https://www.fantasypros.com/nfl/projections/wr.php?week=draft&scoring=PPR&week=draft"
TE = "https://www.fantasypros.com/nfl/projections/te.php?week=draft&scoring=PPR&week=draft"
FLEX = "https://www.fantasypros.com/nfl/projections/flex.php?week=draft&scoring=PPR&week=draft"
K = "https://www.fantasypros.com/nfl/projections/k.php?week=draft"
DEF = "https://www.fantasypros.com/nfl/projections/dst.php?week=draft"

urls = [QB, RB, WR, TE, FLEX, K, DEF]

In [3]:
# # Make a get request
# r = requests.get(url)

# # Initialize soup object
# soup = BeautifulSoup(r.text, "html.parser")

In [4]:
# # Find the table
# table = soup.find("table")

In [5]:
# # print(table)

# # Extract table data

# # Extract headers
# headers = []
# for header in table.find_all("th"):
#     headers.append(header.get_text().strip())


# # Extract rows
# rows = []
# for row in table.find_all("tr"):
#     cells = row.find_all("td")
#     cells = [cell.get_text().strip() for cell in cells]
#     if cells:
#         rows.append(cells)

In [6]:
# # Check if we can see the data

# for header in headers:
#     print(header, end=" | ")
# # print('\n' + '-'*40)

# for row in rows:
#     print(" | ".join(row))

In [8]:
# print(headers)
# print(rows[1:])

In [9]:
# # Save data as csv

# rows = rows[1:]

# with open("rb_projections", "w") as file:
#     writer = csv.writer(file)
#     writer.writerow(headers)
#     for row in rows:
#         writer.writerow(row)

In [10]:
# data = pd.read_csv("rb_projections")

# data.head(25)

# # data.dtypes

# data["YDS"] = data["YDS"].str.replace(",", "")
# data["YDS"] = pd.to_numeric(data["YDS"], errors="coerce")
# data["YDS"] = data["YDS"].astype("float64")

# data.head()

In [11]:
# # Overwrite the csv file with the new cleaned data

# data.to_csv("rb_projections", index=False)

NameError: name 'data' is not defined